In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
import numpy as np

In [5]:
classes = pd.read_csv("ClassData.csv")
sales = pd.read_csv("SalesData.csv")
cart = pd.read_csv("CartData.csv")
user = pd.read_csv("UserData.csv")
attendance = pd.read_csv('AttendanceData.csv')

In [6]:
classes_df = classes.copy()
sales_df = sales.copy()
attendance_df = attendance.copy()
cart_df = cart.copy()
user_df = user.copy()

In [7]:
classes_df['classstartdatetime'] = pd.to_datetime(classes_df['classstartdate'] + ' ' + classes_df['classstarttime'], format='%m/%d/%y %H:%M:%S', errors='coerce')

sales_df['orderdatetime'] = pd.to_datetime(sales_df['orderdate'] + ' ' + sales_df['order time'], format='%m/%d/%y %H:%M:%S', errors='coerce')

cart_df['cartadddatetime'] = pd.to_datetime(cart_df['shoppingcartadddate'] + ' ' + cart_df['shoppingcartaddtime'], format='%m/%d/%y %H:%M:%S', errors='coerce')

# For the user's date of birth (DOB), a different format, possibly 'MM/DD/YY'
user_df['userdob'] = pd.to_datetime(user_df['userdob'], format='%m/%d/%y', errors='coerce')

In [30]:
classes_df

,classid,classstarttime,classendtime,classteacher,classcost,classstartdate,classtag,classstartdatetime
0,42.0,19:30:00,21:30:00,Robert,16.0,8/23/23,Dance,2023-08-23 19:30:00
1,43.0,13:00:00,15:00:00,Peter,16.0,8/26/23,Dance,2023-08-26 13:00:00
2,45.0,13:00:00,15:00:00,Peter,16.0,8/27/23,Dance,2023-08-27 13:00:00
3,46.0,15:30:00,17:30:00,Robert,16.0,8/27/23,Dance,2023-08-27 15:30:00
4,47.0,19:30:00,21:00:00,Robert,12.0,8/24/23,Pilates,2023-08-24 19:30:00
...,...,...,...,...,...,...,...,...
121,187.0,13:00:00,14:15:00,Mary,10.0,12/2/23,Yoga Beginner,2023-12-02 13:00:00
122,188.0,19:30:00,21:00:00,Mary,12.0,12/1/23,Pilates,2023-12-01 19:30:00
123,189.0,19:30:00,21:00:00,Mary,12.0,12/8/23,Pilates,2023-12-08 19:30:00
124,190.0,13:00:00,14:15:00,Mary,10.0,12/16/23,Yoga Beginner,2023-12-16 13:00:00


In [8]:
# 2. Merge the dataframes
merged_data = attendance_df.merge(classes_df, on='classid', how='left')
merged_data = merged_data.merge(sales_df, on=['salesorderid', 'userid'], how='left')
merged_data = merged_data.merge(user_df, on='userid', how='left')

In [43]:
# 3. Aggregate cart data for a new feature: `cart_adds`
cart_counts = cart_df.groupby('classid').size().reset_index(name='cart_adds')
merged_data = merged_data.merge(cart_counts, on='classid', how='left')
merged_data['cart_adds'] = merged_data['cart_adds'].fillna(0) # Fill missing values with 0
print(cart_counts)

     classid  cart_adds
0         42          3
1         43         15
2         45          4
3         46          8
4         47          5
..       ...        ...
121      184          1
122      185          1
123      187          7
124      188          1
125      190          1

[126 rows x 2 columns]


In [10]:
# 4. Create additional features from merged data
merged_data['time_until_class'] = (merged_data['classstartdatetime'] - merged_data['orderdatetime']).dt.total_seconds() / 3600 / 24 # in days
merged_data['user_age'] = (pd.Timestamp.now() - merged_data['userdob']).dt.total_seconds() / (365.25 * 24 * 3600) # in years

In [11]:
# 5. Aggregate to the class level to get our target variable
class_demand = merged_data.groupby('classid').agg(
    total_enrollments=('enrollmentid', 'size'),
    avg_price=('totalamountpaid', 'mean'),
    base_cost=('classcost', 'first'),
    time_of_day=('classstarttime', 'first'),
    day_of_week=('classstartdate', lambda x: pd.to_datetime(x.iloc[0]).dayofweek),
    teacher=('classteacher', 'first'),
    cart_interest=('cart_adds', 'first'),
    # Note: For user features, we can't aggregate directly. We'd have to use a user-centric model or choose a representative value (e.g., average age of enrollees)
    # For now, i'd keep the user features out of this aggregated model.
).reset_index()

In [23]:
threshold = 2
class_demand['demand_level'] = class_demand['total_enrollments'].apply(lambda x: 'high' if x > threshold else 'normal')

In [24]:
class_demand.isna().sum()

classid              0
total_enrollments    0
avg_price            0
base_cost            0
time_of_day          0
day_of_week          0
teacher              0
cart_interest        0
demand_level         0
dtype: int64

In [25]:
# 1. Handle the numerical column `base_cost` by imputing with the median
median_base_cost = class_demand['base_cost'].median()
class_demand['base_cost'].fillna(median_base_cost, inplace=True)

# 2. Handle the categorical columns `time_of_day`, `day_of_week`, and `teacher` by creating a new category for the missing values.
class_demand['time_of_day'].fillna('Unknown Time', inplace=True)
class_demand['day_of_week'].fillna('Unknown Day', inplace=True)
class_demand['teacher'].fillna('Unknown Teacher', inplace=True)

# 3. Double-check for any remaining missing values
print(class_demand.isnull().sum())

classid              0
total_enrollments    0
avg_price            0
base_cost            0
time_of_day          0
day_of_week          0
teacher              0
cart_interest        0
demand_level         0
dtype: int64


/var/folders/01/4xc8q_m12ngb83yl1h4c6hym0000gn/T/ipykernel_84144/546780942.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  class_demand['base_cost'].fillna(median_base_cost, inplace=True)
/var/folders/01/4xc8q_m12ngb83yl1h4c6hym0000gn/T/ipykernel_84144/546780942.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

In [26]:
class_demand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   classid            115 non-null    int64  
 1   total_enrollments  115 non-null    int64  
 2   avg_price          115 non-null    float64
 3   base_cost          115 non-null    float64
 4   time_of_day        115 non-null    object 
 5   day_of_week        115 non-null    object 
 6   teacher            115 non-null    object 
 7   cart_interest      115 non-null    int64  
 8   demand_level       115 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 8.2+ KB


In [33]:
# Identify features and target
X = class_demand.drop('demand_level', axis=1).copy()
y = class_demand['demand_level'].copy()

# Ensure all categorical columns are of string type
for col in ['time_of_day', 'day_of_week', 'teacher']:
    X[col] = X[col].astype(str)

# Define feature types based on column names
categorical_features = ['time_of_day', 'day_of_week', 'teacher']
numerical_features = ['base_cost', 'cart_interest']

# Create the preprocessor using the defined lists of columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop'
)

# Create the imbalanced-learn pipeline
# The order is: preprocessor -> smote -> classifier
model = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42, k_neighbors=3)),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'))
])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Fit the pipeline on the training data
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        high       0.94      0.89      0.91        18
      normal       0.67      0.80      0.73         5

    accuracy                           0.87        23
   macro avg       0.80      0.84      0.82        23
weighted avg       0.88      0.87      0.87        23



In [31]:
def set_dynamic_price(model, class_data, base_price):
    """
    Predicts demand and sets the new price.
    """
    # Create a DataFrame from new class data for prediction
    new_class_data_df = pd.DataFrame([class_data])

    # Predict the demand using the trained pipeline
    # The pipeline automatically handles preprocessing.
    predicted_demand = model.predict(new_class_data_df)[0]

    # Apply pricing rules
    if predicted_demand == 'high':
        new_price = base_price * 1.15  # 15% price increase
        print(f"High demand predicted. Price increased by 15% from ${base_price} to ${new_price:.2f}.")
    elif predicted_demand == 'normal':
        new_price = base_price * 0.90 # 10% price decrease
        print(f"Normal demand predicted. Price decreased by 10% from ${base_price} to ${new_price:.2f}.")
    else:
        new_price = base_price
        print(f"Unknown demand predicted. Price remains at ${base_price:.2f}.")

    return new_price

# Example of a new class
new_class_features = {
    'base_cost': 65.0,
    'cart_interest': 25,
    'time_of_day': '17:00:00',
    'day_of_week': 'Fri',
    'teacher': 'C'
}

# Get the dynamic price recommendation
dynamic_price = set_dynamic_price(model, new_class_features, new_class_features['base_cost'])
print(f"The recommended price for the new class is ${dynamic_price:.2f}.")

High demand predicted. Price increased by 15% from $65.0 to $74.75.
The recommended price for the new class is $74.75.


In [39]:
# Identify features and target
X = class_demand.drop(columns=['demand_level','total_enrollments'], axis=1).copy()
y = class_demand['total_enrollments'].copy()

# Ensure all categorical columns are of string type
for col in ['time_of_day', 'day_of_week', 'teacher']:
    X[col] = X[col].astype(str)

# Define feature types based on column names
categorical_features = ['time_of_day', 'day_of_week', 'teacher']
numerical_features = ['base_cost', 'cart_interest']

# Create the preprocessor using the defined lists of columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop'
)
# Create and train a regression model
regression_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regression_model.fit(X_train, y_train)

# Evaluate the model
y_pred = regression_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred) # No 'squared' parameter
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Root Mean Squared Error (RMSE): 1.86


In [40]:
# Assume a new class with the following features
new_class_data = {
    'time_of_day': '17:00:00',
    'day_of_week': '4', # Friday
    'teacher': 'Teacher B',
    'cart_interest': 25
}

# Define a range of possible prices to test
price_range = range(40, 101, 5)

# Simulate revenue for each price point
simulated_revenue = []
for price in price_range:
    # Create a test row with the new price
    test_data = new_class_data.copy()
    test_data['base_cost'] = price
    test_data_df = pd.DataFrame([test_data])

    # Predict the number of enrollments for this price
    predicted_enrollments = regression_model.predict(test_data_df)[0]

    # Calculate the simulated revenue
    revenue = predicted_enrollments * price
    simulated_revenue.append({'price': price, 'predicted_enrollments': predicted_enrollments, 'revenue': revenue})

# Convert to a DataFrame for easy analysis
revenue_df = pd.DataFrame(simulated_revenue)

# Find the price that maximizes revenue
optimal_price_row = revenue_df.loc[revenue_df['revenue'].idxmax()]

print(revenue_df)
print("\n--- Optimal Price Found ---")
print(f"Optimal Price: ${optimal_price_row['price']:.2f}")
print(f"Predicted Enrollments at this Price: {optimal_price_row['predicted_enrollments']:.2f}")
print(f"Maximized Revenue: ${optimal_price_row['revenue']:.2f}")

    price  predicted_enrollments  revenue
0      40                  15.71   628.40
1      45                  15.71   706.95
2      50                  15.71   785.50
3      55                  15.71   864.05
4      60                  15.71   942.60
5      65                  15.71  1021.15
6      70                  15.71  1099.70
7      75                  15.71  1178.25
8      80                  15.71  1256.80
9      85                  15.71  1335.35
10     90                  15.71  1413.90
11     95                  15.71  1492.45
12    100                  15.71  1571.00

--- Optimal Price Found ---
Optimal Price: $100.00
Predicted Enrollments at this Price: 15.71
Maximized Revenue: $1571.00


In [41]:
# Assuming 'class_demand', 'preprocessor', and 'regression_model' are already defined.

# Create a copy for simulation
simulation_data = class_demand.copy()

# Predict enrollments for each class using its original base_cost
# The 'regression_model' takes the DataFrame and predicts the 'total_enrollments'
simulation_data['predicted_enrollments'] = regression_model.predict(simulation_data)

# Calculate the simulated revenue for each class
simulation_data['simulated_revenue'] = simulation_data['predicted_enrollments'] * simulation_data['base_cost']

# Compare the total simulated revenue to the total original revenue
total_original_revenue = (simulation_data['total_enrollments'] * simulation_data['base_cost']).sum()
total_simulated_revenue = simulation_data['simulated_revenue'].sum()

print(f"Total Original Revenue: ${total_original_revenue:.2f}")
print(f"Total Simulated Revenue: ${total_simulated_revenue:.2f}")
print(f"Difference: ${total_simulated_revenue - total_original_revenue:.2f}")

# Compare the predicted enrollments to the actual ones
print("\nPredicted vs. Actual Enrollments:")
print(simulation_data[['classid', 'total_enrollments', 'predicted_enrollments']])

Total Original Revenue: $11151.00
Total Simulated Revenue: $11259.39
Difference: $108.39

Predicted vs. Actual Enrollments:
     classid  total_enrollments  predicted_enrollments
0         42                  1               1.798000
1         43                 11               8.672000
2         45                  3               2.000000
3         46                  4               4.063000
4         47                  3               3.364667
..       ...                ...                    ...
110      182                  1               1.100000
111      184                  1               1.030000
112      185                  1               1.100000
113      187                  5               4.062333
114      190                  1               1.030000

[115 rows x 3 columns]
